In [ ]:
from dataset import *
from network import *

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
gpu_devices = list(np.arange(torch.cuda.device_count()))
multi_gpu = len(gpu_devices) > 1

output_folder = r'./Outputs/pred/DAFNet/EORSSD/Test'
ckpt_folder = r'./Checkpoints'
dataset_root = r'../Dataset/EORSSD'

In [ ]:
test_loader = data.DataLoader(EORSSD(dataset_root, 'test', aug=False), shuffle=False, batch_size=1)

In [ ]:
net = DAFNet(return_loss=False, init_path=os.path.join(ckpt_folder, 'warehouse')).eval().cuda()
net.load_state_dict(torch.load(os.path.join(ckpt_folder, 'trained', 'trained.pth')))

In [ ]:
import os
os.system('rm -rf ./Outputs/pred/DAFNet/EORSSD/Test/*.png')

In [ ]:
infer_time = 0
num_test = 0
for image, label, edge, prefix in tqdm(test_loader):
    num_test += 1
    with torch.no_grad():
        image, label, edge = image.cuda(), label.cuda(), edge.cuda()
        B = image.size(0)
        t1 = time.time()
        smap, _ = net(image, label, edge)
        t2 = time.time()
        infer_time += (t2 - t1)
        for b in range(B):
            path = os.path.join(output_folder, prefix[b] + '.png')
            save_smap(smap[b, ...], path)
infer_time /= num_test
print('average inference speed: {} FPS'.format(int(np.round(1 / infer_time))))